In [7]:
import os
import random
import PIL
import cv2
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

In [8]:
def make_square(im, min_size=2600, fill_color=(255, 255, 255)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [9]:
class Augmentation:
    
    def __init__(self, path):
        self.path = path
    
    def rotate_images(self, op, iterations):
        self.op = op
        self.iterations = iterations
        with open('responce.json', 'r') as f:
          data = json.load(f)
        os.mkdir(self.op)
        print(f"created {self.op} directory")
        images_list = []
        
        
        for k in range(0, self.iterations):
            print(f"iteration {k}")
            for f in os.listdir(self.path):
                json_dict = {}
                angle = random.randint(-20, 20)
                im = Image.open(os.path.join(self.path, f))
                im = im.rotate(angle, PIL.Image.NEAREST, expand = 1, fillcolor = 'white')
                im = make_square(im)
                im = ImageOps.grayscale(im)
                im = cv2.resize(np.array(im), (1024, 1024), interpolation = cv2.INTER_AREA)
                im_name = 'Image-' + str(k) + '-' + str(angle) + f
                json_dict['id'] = im_name
                json_dict['angle'] = angle + data[self.path+f]
                cv2.imwrite(os.path.join(self.op, im_name), im)
                images_list.append(json_dict)
        
        return images_list


In [10]:
augmetation = Augmentation('invoices_rotated/')
images_lst = augmetation.rotate_images('dataset_highres/', 10)

created dataset_highres/ directory
iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [11]:
df = pd.DataFrame.from_dict(images_lst)
df.to_csv('labels_highres.csv', index=False)
df

,id,angle
0,Image-0--50.png,2
1,Image-0--191.png,-21
2,Image-0-510.png,1
3,Image-0-92.png,0
4,Image-0-23.png,-5
...,...,...
105,Image-9--15.png,-7
106,Image-9--126.png,-7
107,Image-9-07.png,-2
108,Image-9-38.png,-1
